I'm pretty limited for this figure, because I drew most of it by hand in inkscape, which isn't always the best idea. Nevertheless, I can atleast go through the process of generating the data for that figure.

# Libraries

In [1]:
%load_ext rpy2.ipython

In [2]:
%%R 
library(tidyverse)
library(readxl)

R[write to console]: ── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

R[write to console]: ✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   2.1.1     ✔ forcats 0.5.1

R[write to console]: ── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



# Data

In [3]:
%%R
best_hits <- read_csv("data/Ecoli_pangenome_best_hits.csv")

Rows: 7262 Columns: 25
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (8): qseqid, sseqid, component, cinful_id, pephash, sample, contig, seq
dbl (13): pident, length, mismatch, gapopen, qstart, qend, sstart, send, eva...
lgl  (4): hmmerHit, verified, allStandardAA, signalMatch

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [4]:
%%R
assemblySummary_ENAInformation <- read_excel("data/Ecoli_assemblySummary.xlsx", sheet="ENA-Information")

In [6]:
%%R -o assemblySummary_SequencedGenomes
assemblySummary_SequencedGenomes <- read_excel("data/Ecoli_assemblySummary.xlsx", sheet="Sequenced-Genomes", skip=5) %>%
	separate("Sequence_Name",c("phylogroup")) %>%
	select(c("Genome_ID","phylogroup","Isolation source","Strain Category")) %>%
	rename("GEMBASE_NAME"="Genome_ID") %>%
	drop_na()


R[write to console]: New names:
* `` -> ...8
* `` -> ...20
* `` -> ...28
* `` -> ...35



In [7]:
%%R -o assemblySummary_ENAInformation_phylogroup
assemblySummary_ENAInformation_phylogroup <- assemblySummary_ENAInformation %>% 
	full_join(assemblySummary_SequencedGenomes) %>%
	drop_na()


Joining, by = "GEMBASE_NAME"


In [8]:
%%R -o best_hits_assemblyID
best_hits_assemblyID <- best_hits %>%
	separate(cinful_id,c(NA, "ASSEMBLY_ACC_ID"), sep="/") %>%
	separate(ASSEMBLY_ACC_ID, c("ASSEMBLY_ACC"), sep = "[.]")

In [9]:
%%R -o best_hits_phylogroup
best_hits_phylogroup <- best_hits_assemblyID %>%
	full_join(assemblySummary_ENAInformation_phylogroup) 

Joining, by = "ASSEMBLY_ACC"


In [10]:
%%R -o microcin_hits_perPhylogroup
microcin_hits_perPhylogroup <- best_hits_phylogroup %>%
	filter(component == "microcins.verified") %>%
	group_by(sseqid, phylogroup) %>%
	summarise(n = n())


`summarise()` has grouped output by 'sseqid'. You can override using the `.groups` argument.


In [20]:
%%R 
microcin_hits_perPhylogroup %>%
	pivot_wider(values_from=n, names_from=phylogroup)

# A tibble: 9 × 9
# Groups:   sseqid [9]
  sseqid                        B2     F     A    B1     C     D  H299     E
  <chr>                      <int> <int> <int> <int> <int> <int> <int> <int>
1 E492_sp_Q9Z4N4_MCEA_KLEPN      2     1    NA    NA    NA    NA    NA    NA
2 H47_sp_P62530_MCHB_ECOLX      24     1     1     5     1     6     3    NA
3 I47_tr_Q712Q0_Q712Q0_ECOLX     5     1     3     5     1     6     3    NA
4 L_tr_Q841V4_Q841V4_ECOLX       4    NA     3     2    NA     3    NA     2
5 M_tr_Q83TS1_Q83TS1_ECOLX      39     2     7     6    NA     9     3     3
6 N_tr_C3VUZ5_C3VUZ5_ECOLX       1    NA    NA     1    NA    NA    NA    NA
7 PDI_tr_I6ZU90_I6ZU90_ECOLX     5     2     6     7     3     6    NA     1
8 S_tr_H9ZMG7_H9ZMG7_ECOLX       1    NA    NA     4    NA     1     1     1
9 V_sp_P22522_CEAV_ECOLX        87    12    37    37    10    10    17     5


So that's where the data comes from. I just took the known phylogenetic relationships of the verified micrcoins, as well as the known relationships between the phylogroups and hand drew dendrograms such that each axis is phylogenetically sorted. Then the boxes were colored by the number, and left blank f NA

Now we can do a little statistical assesment of how the phylogroups aren't equally represented in this analysis

In [12]:
%%R
n_assembliesPhylogroup <- assemblySummary_ENAInformation_phylogroup %>%
	count(phylogroup, name="n_assemblies") %>%
	mutate(freq_assemblies = n_assemblies / sum(n_assemblies))
n_assembliesPhylogroup

# A tibble: 8 × 3
  phylogroup n_assemblies freq_assemblies
  <chr>             <int>           <dbl>
1 A                   292          0.239 
2 B1                  276          0.225 
3 B2                  305          0.249 
4 C                    16          0.0131
5 D                   174          0.142 
6 E                    59          0.0482
7 F                    70          0.0572
8 H299                 32          0.0261


In [13]:
%%R
best_hits_phylogroup_expected <- best_hits_phylogroup %>%
	filter(component == "microcins.verified") %>%
	group_by(phylogroup) %>%
	summarise(n_microcins = n()) %>%
	full_join(n_assembliesPhylogroup) %>%
	mutate(expected_microcins = sum(n_microcins) * freq_assemblies)
best_hits_phylogroup_expected

Joining, by = "phylogroup"
# A tibble: 8 × 5
  phylogroup n_microcins n_assemblies freq_assemblies expected_microcins
  <chr>            <int>        <int>           <dbl>              <dbl>
1 A                   57          292          0.239               96.9 
2 B1                  67          276          0.225               91.5 
3 B2                 168          305          0.249              101.  
4 C                   15           16          0.0131               5.31
5 D                   41          174          0.142               57.7 
6 E                   12           59          0.0482              19.6 
7 F                   19           70          0.0572              23.2 
8 H299                27           32          0.0261              10.6 


In [14]:
%%R
chisq.test(x = best_hits_phylogroup_expected$n_assemblies)


	Chi-squared test for given probabilities

data:  best_hits_phylogroup_expected$n_assemblies
X-squared = 700.2, df = 7, p-value < 2.2e-16



In [15]:
%%R
chisq.test(x = best_hits_phylogroup_expected$n_microcins, p=best_hits_phylogroup_expected$freq_assemblies)


	Chi-squared test for given probabilities

data:  best_hits_phylogroup_expected$n_microcins
X-squared = 118.67, df = 7, p-value < 2.2e-16

